In [25]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config
from datetime import datetime, date
import time
import pygrib
import pandas as pd
from dateutil.relativedelta import relativedelta

ct_df = pd.read_csv("usregions.csv")[:30000]
lls = list(zip(ct_df['lat'], ct_df['lng']))
last_report = pd.read_csv("report.csv").to_dict()
report_dict = {
        "Date": [(date.today()+relativedelta(days=i)).strftime("%a, %d %b %Y") for i in range(15)],
        "Current FC": [],
        "FC 6 hours ago": last_report['Current FC'],
        "FC 12 hours ago": last_report['FC 6 hours ago'],
        "FC 18 hours ago": last_report['FC 12 hours ago'],
        "FC 24 hours ago": last_report['FC 18 hours ago'],
        "Diff 12 hours ago": [],
        "Diff 24 hours ago": []
    }
dt = datetime.now()
file_dt = "".join(list(map(str,[dt.year,dt.month,f"{dt.day:02d}"])))

client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket = client.list_objects(Bucket='noaa-gefs-pds', Prefix = "gefs."+file_dt+"/", Delimiter="/",)

while True:
    if bucket.get('CommonPrefixes') is not None:
        fldr = bucket.get('CommonPrefixes')[-1].get('Prefix')
        break
    else:
        time.sleep(300)

cycle = fldr.split('/')[-2]
base = f'{fldr[:-1]}/atmos/pgrb2ap5/gec00.t{cycle}z.pgrb2a.0p50.f'
files = []
for i in range(0,243,3):
    files.append(base+f"{i:03d}")
for i in range(246,390,6):
    files.append(base+f"{i}")


def process(file, cycle):
    grbs = pygrib.open(file)
    grb = grbs.select(name='Temperature')[-1]
    data = grb.data()[0]
    
    def f(lat, lon):
        return round(abs(data[int(round(-2*lat)+180),int(round(2*lon))]*1.8-459.67-65),2)
    
    ct_df['dd_'+cycle] = [f(lat, lon) for lat,lon in lls]
    
    grbs.close()
def _to_state(x):
    return pd.Series({'dd':(x['population'] * x['dd']/x['population'].sum()).sum(),'population': x['population'].sum()})
def _to_region(x):
    return pd.Series({'dd':(x['population'] * x['dd']/x['population'].sum()).sum(),'population': x['population'].sum()})
while files:
    idx = int(files[0].split(".")[-1][-3:])

    print(files[0])
    try:
        if idx%24==0 and idx>0:
            ct_df['dd'] = (ct_df['dd_00']+ct_df['dd_03']+ct_df['dd_06']+ct_df['dd_09']+ct_df['dd_12']+ct_df['dd_15']+ct_df['dd_18']+ct_df['dd_21'])/8
            state_df = ct_df.groupby(['state_name','region'])[['population', 'dd']].apply(_to_state).reset_index()
            region_df = state_df.groupby(['region'])[['population','dd']].apply(_to_region).reset_index()
            national_dd = round((region_df['population']*region_df['dd']).sum()/region_df['population'].sum(),2)
            report_dict['Current FC'].append(national_dd)
            
            report_df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in report_dict.items() ]))
            report_df['Diff 12 hours ago'] = report_df['Current FC']-report_df['FC 12 hours ago']
            report_df['Diff 24 hours ago'] = report_df['Current FC']-report_df['FC 24 hours ago']
            report_df.to_csv('report.csv')
        client.download_file('noaa-gefs-pds', files[0], 'temp')
        process('temp', f"{idx%24:02d}")
        
    except Exception as e:
        print(e)
        # time.sleep(60)
        break
    del files[0]

    


gefs.20231111/06/atmos/pgrb2ap5/gec00.t06z.pgrb2a.0p50.f000
An error occurred (404) when calling the HeadObject operation: Not Found


In [24]:
report_df

,Date,Current FC,FC 6 hours ago,FC 12 hours ago,FC 18 hours ago,FC 24 hours ago,Diff 12 hours ago,Diff 24 hours ago
0,"Sat, 11 Nov 2023",14.64,NaN,NaN,NaN,NaN,NaN,NaN
1,"Sun, 12 Nov 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Mon, 13 Nov 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Tue, 14 Nov 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Wed, 15 Nov 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Thu, 16 Nov 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,"Fri, 17 Nov 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Sat, 18 Nov 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Sun, 19 Nov 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Mon, 20 Nov 2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
ct_df = pd.read_csv("uscities.csv")
states_to_regions = {"Washington": "Pacific", "Oregon":"Pacific","California":"Pacific","Nevada":"Mountain","Wyoming":"Mountain","Arizona":"Mountain","Montana":"Mountain","Idaho":"Mountain","Utah":"Mountain","Colorado":"Mountain","New Mexico":"Mountain","North Dakota":"Midwest","South Dakota":"Midwest","Nebraska":"Midwest","Kansas":"Midwest","Minnesota":"Midwest","Iowa":"Midwest","Missouri":"Midwest","Wisconsin":"Midwest","Michigan":"Midwest","Illinois":"Midwest","Indiana":"Midwest","Ohio":"Midwest","New York":"Northeast","Pennsylvania":"Northeast","New Jersey":"Northeast","Vermont":"Northeast","New Hampshire":"Northeast","Maine":"Northeast","Massachusetts":"Northeast","Rhode Island":"Northeast","Connecticut":"Northeast","Texas":"South","Oklahoma":"South","Arkansas":"South","Louisiana":"South","North Carolina":"South","South Carolina":"South","Florida":"South","Georgia":"South","West Virginia":"South","Virginia":"South","Delaware":"South","Maryland":"South","Tennessee":"South","Mississippi":"South","Alabama":"South","Kentucky":"South","District of Columbia":"South","Puerto Rico":"lol","Hawaii":"lol","Alaska":"lol"}


In [22]:
ct_df["region"] =[states_to_regions[x] for x in ct_df["state_name"]]

ct_df.drop(ct_df[ct_df["region"]=="lol"].index, inplace=True)

In [23]:
ct_df.to_csv("usregions.csv")